### Search Engine With tools and Agents

In [7]:
# Arxiv -- Research
## tools creation
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper


In [8]:
# used the inbuilt tool of wikipedia
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name


'wikipedia'

In [9]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name

'arxiv'

In [10]:
tools=[wiki,arxiv]


In [11]:
## Custom tools[RAG tool]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter



USER_AGENT environment variable not set, consider setting it to identify your requests.


In [12]:
loader=WebBaseLoader("https://docs.langchain.com/langsmith/home")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)

# Using a lighter model for faster processing
embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2",  
    model_kwargs={'device': 'cpu'}
)

vectordb=FAISS.from_documents(documents, embeddings)
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000014CD2F66CF0>, search_kwargs={})

In [13]:
from langchain_core.tools import tool

@tool
def langsmith_search(query: str) -> str:
    """Search for information about LangSmith."""
    docs = retriever.invoke(query)
    return "\n\n".join([doc.page_content for doc in docs])

retriever_tool = langsmith_search
retriever_tool.name

'langsmith_search'

In [14]:
tools=[wiki,arxiv,retriever_tool]


In [15]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'e:\\Gen-ai-projects\\genvenv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 StructuredTool(name='langsmith_search', description='Search for information about LangSmith.', args_schema=<class 'langchain_core.utils.pydantic.langsmith_search'>, func=<function langsmith_search at 0x0000014CD3136980>)]

In [16]:
### run all these tools with agents and llm models
## tools,llm-->agentExecutor
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from pydantic import SecretStr
import os
from dotenv import load_dotenv
load_dotenv()
groq_api_key=os.getenv("GROQ_API_KEY","")
openai_api_key = os.getenv("OPENROUTER_API_KEY","")

llm1 = ChatOpenAI(
    model="tngtech/tng-r1t-chimera:free",
    api_key=SecretStr(openai_api_key),
    base_url="https://openrouter.ai/api/v1",
    temperature=0.7
)
llm2=ChatGroq(
    api_key=SecretStr(groq_api_key),
    model="qwen/qwen3-32b"
)

In [17]:
## prompt template - using ChatPromptTemplate instead of hub
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Use the provided tools to answer questions."),
    MessagesPlaceholder("chat_history", optional=True),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),
])

In [22]:
## Create agent using LangGraph's create_react_agent
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm2, tools)

C:\Users\ayang\AppData\Local\Temp\ipykernel_7852\1608783035.py:4: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent_executor = create_react_agent(llm2, tools)


In [23]:
## Test the agent
result = agent_executor.invoke({"messages": [("user", "Tell me about LangSmith")]})
print(result["messages"][-1].content)

LangSmith is a comprehensive platform for developing, debugging, and deploying **Large Language Model (LLM) applications**. Here's a concise overview:

### Key Features:
1. **Framework-Agnostic**: Works with or without LangChain’s open-source libraries (`langchain` and `langgraph`).
2. **Integrated Workflow**:
   - **Observability**: Trace requests and debug applications.
   - **Evaluation**: Measure and track AI output quality over time.
   - **Deployment**: Manage deployments and scale agents in production.
3. **Development Tools**:
   - **Agent Builder (Beta)**: No-code interface for designing and deploying AI agents.
   - **Studio**: Visual interface for building, testing, and refining LLM applications end-to-end.
   - **Prompt Engineering**: Test and iterate on prompts with versioning and collaboration.

4. **Security & Compliance**:
   - Meets **HIPAA, SOC 2 Type 2, and GDPR** standards.
   - Secure data management and access control.

5. **Flexibility**:
   - Deploy in **managed

In [24]:
## Test with Arxiv
result = agent_executor.invoke({"messages": [("user", "Find recent papers about transformers in AI")]})
print(result["messages"][-1].content)

Here's a recent paper about transformers in AI from arXiv:

**Title:** Foundations of GenIR  
**Authors:** Qingyao Ai, Jingtao Zhan, Yiqun Liu  
**Published:** 2025-01-06  

**Summary:**  
This paper explores how modern generative AI models (like transformers) are transforming information access systems. It contrasts traditional AI approaches with the new paradigm of generative models, addressing challenges in knowledge distillation, retrieval-augmented generation, and the integration of large language models into retrieval systems.

Would you like me to expand on any specific aspects of this paper?


In [25]:
## Test with Wikipedia
result = agent_executor.invoke({"messages": [("user", "What is machine learning?")]})
print(result["messages"][-1].content)

Machine learning (ML) is a field of study within artificial intelligence (AI) focused on developing algorithms that enable computers to learn patterns and make decisions from data. These algorithms improve their performance on tasks over time without being explicitly programmed for specific outcomes. Key aspects include:

1. **Learning from Data**: ML models identify patterns in historical or input data to make predictions or decisions.
2. **Generalization**: They adapt to new, unseen data by applying learned patterns rather than following rigid rules.
3. **Statistical Foundations**: Most methods rely on statistical techniques to optimize performance and minimize errors.
4. **Applications**: Used in diverse fields like image recognition, natural language processing, finance, healthcare, and more.

For example, a machine learning model trained on thousands of cat and dog photos can later classify new images as "cat" or "dog" with high accuracy. The field continues to evolve with advance